# Basic usage of ABCD database

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from ase.io import iread, read
# from utils.ext_xyz import XYZReader

from abcd import ABCD

First of all, we need to define the url of the database. It could be local or remote:

- direct access: url = 'mongodb://localhost:27017'
- api access: url = 'http://localhost/api'

using with statement to catch the riased exceptions. You may can ignore them  but in that case need to handle all the unexpected events. (cannot connect to db, lost connection, wrong filter, wrong url, etc. )

In [3]:
url = 'mongodb://localhost:27017/'
abcd =  ABCD(url, db_name='abcd', collection_name='atoms')

print(abcd)

MongoDatabase(url=localhost:27017, db=abcd, collection=atoms)


In [10]:
abcd.print_info()

================== ABCD MongoDB ==================
      type: mongodb
      host: localhost
      port: 27017
        db: abcd
collection: atoms
number of confs: 14


## Cleanup 

WARNING!! Remove all elements from the database.
Only supported in the case of local access

In [7]:
with abcd as db:
    db.destroy()

## Uploading configurations

In [8]:


direcotry = Path('data/')
file = direcotry / 'bcc_bulk_54_expanded_2_high.xyz'
# file = direcotry / 'GAP_1.xyz'

Uploading configurations on-by-one directly from an ase atoms object:

In [9]:
%%time
with abcd as db:

    for atoms in iread(file.as_posix(), index=slice(None)):
        
        # Hack to fix the representation of forces
        
        atoms.calc.results['forces'] = atoms.arrays['force']
        del(atoms.arrays['force'])
            
        db.push(atoms)
    

CPU times: user 112 ms, sys: 4.49 ms, total: 117 ms
Wall time: 174 ms


Reading the trajectory from file:

In [11]:
%%time
traj = read(file.as_posix(), index=slice(None))
len(traj)

CPU times: user 15.1 ms, sys: 1.48 ms, total: 16.6 ms
Wall time: 16.1 ms


In [12]:
%%time
with XYZReader(file) as reader:
    traj = list(reader.read_atoms(forces_label='force'))


NameError: name 'XYZReader' is not defined

Pushing the whole trajectory to the database:

In [13]:
%%time
db.push(traj)

AttributeError: type object 'AtomsModel' has no attribute 'insert'

Uploading a whole file and injecting to the database on the server side:

In [15]:
%%time
with abcd as db:
    db.upload(file.as_posix())

AttributeError: type object 'AtomsModel' has no attribute 'insert'

An alternative way to upload file to database:

In [ ]:
%%time
with abcd as db, XYZReader(file) as reader:
    db.push(reader.read_atoms(forces_label='force'))

In [ ]:
abcd.info()

## Uploading


In [ ]:
for file in Path('data/').glob('*.xyz'):
    print(file)
#     with abcd as db, XYZReader(file) as reader:
#         db.push(reader.read_atoms())
        
    traj = read(file.as_posix(), index=slice(None))
    db.push(traj)

In [ ]:
for file in Path('GB_alphaFe_001/tilt/').glob('*.xyz'):
    print(file)
    gb_params = {
        'name': 'alphaFe',
        'type': 'tilt',
        'params': file.name[:-4]
        
    }    

    traj = read(file.as_posix(), index=slice(None))
    db.push(traj, extra_info={'GB_params': gb_params})

In [ ]:
#     with abcd as db, XYZReader(file) as reader:
#         db.push(reader.read_atoms(), extra_info={'GB_params': gb_params})
